In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
import os

## Combinación de todos los datos

In [ ]:
# Obtén la lista de archivos en la carpeta 'records'
folder_path = 'records'
file_list = os.listdir(folder_path)

# Crea una lista para almacenar los DataFrames de cada archivo
dataframes = []

# Lee cada archivo CSV y añádelo a la lista de DataFrames
for file in file_list:
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        dataframes.append(df)

# Combina todos los DataFrames en uno solo
combined_df = pd.concat(dataframes, ignore_index=True)

# Guarda el DataFrame combinado en un nuevo archivo CSV
combined_df.to_csv('dataset.csv', index=False)

## Dibujado de las clases

In [ ]:
# Carga el archivo CSV combinado
data = pd.read_csv('dataset.csv')

# Separa las características (atributos) de las etiquetas de clase
y = data['action']  # Clases
X = data.drop('action', axis=1)  # Atributos

# Aplica PCA para reducir a tres dimensiones
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

# Crea un DataFrame con los componentes principales y las clases
pca_df = pd.DataFrame(X_pca, columns=['Componente1', 'Componente2', 'Componente3'])
pca_df['Clase'] = y

# Gráfico tridimensional
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Dibuja los puntos en el espacio tridimensional
for clase in pca_df['Clase'].unique():
    class_data = pca_df[pca_df['Clase'] == clase]
    ax.scatter(class_data['Componente1'], class_data['Componente2'], class_data['Componente3'], label=clase)

# Etiquetas y título
ax.set_xlabel('Componente 1')
ax.set_ylabel('Componente 2')
ax.set_zlabel('Componente 3')
ax.set_title('Distribución de Clases en Espacio Tridimensional (PCA) sin clase NONE')

# Leyenda
ax.legend()

plt.show()

## Limpio el dataset

In [ ]:
def clean_data(input_filename, output_filename):
    
    # Carga el archivo CSV 
    data = pd.read_csv(input_filename)
    
    # Elimino ciertos parámetros
    data = data.drop('ray1', axis=1)
    data = data.drop('karty', axis=1)
    data = data.drop('time', axis=1)
    
    ray_columns = ['ray2', 'ray3', 'ray4', 'ray5']
    for column in ray_columns:
        max_value = data[column].max()  # Encuentra el valor máximo en la columna actual
        data[column] = data[column].replace(-1, 10)  # Reemplaza -1 por el valor máximo
        
    # Guardar el dataset limpio en un nuevo archivo CSV
    data.to_csv(output_filename, index=False)
    
clean_data('dataset.csv', 'dataset.csv')

## Normalización de datos

In [ ]:
def normalize_data(input_filename, output_filename):
    
    # Carga del dataset
    data = pd.read_csv(input_filename) 
    
    # Seleccionar las columnas a normalizar
    columns_to_normalize = data.columns[0:data.columns.get_loc('action')]
    
    # Aplicar One-Hot Encoding a la columna 'acción'
    data = pd.get_dummies(data, columns=['action'])
    
    # Reescalo
    scaler = StandardScaler()
    data[columns_to_normalize] = scaler.fit_transform(data[columns_to_normalize])
    
    # Guardar el dataset con One-Hot Encoding en un nuevo archivo CSV
    data.to_csv(output_filename, index=False)

normalize_data('dataset.csv', 'dataset.csv')

## Perceptrón Multicapa

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from ann2 import forward_propagation as n_forward_propagate
from ann2 import backward_propagation as n_backprop
from ann2 import cost as n_cost
from ann2 import converge as n_converge
from ann2 import predict as n_predict

## Cargo el dataset

In [ ]:
def read_data(filename):
    
    # Carga el archivo CSV 
    data = pd.read_csv(filename)
    
    # Encuentra el índice de la columna 'action_ACCELERATE'
    idx = data.columns.get_loc('action_ACCELERATE')

    # Guarda las columnas hasta 'action_ACCELERATE' (excluyendo 'action_ACCELERATE') en una variable X
    X = data.iloc[:, :idx].values

    # Guarda las columnas a partir de 'action_ACCELERATE' en otra variable y
    y = data.iloc[:, idx:].values
    
    return X, y


X, y = read_data('dataset.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
y_test = np.argmax(y_test, axis=1)

## Entreno la red

In [ ]:
def initialize_random_weights(L_in, L_out):
    epsilon_init = 5
    return np.random.rand(L_out, L_in) * 2 * epsilon_init - epsilon_init

# Definir las dimensiones de las capas
hidden_layers_size = [24, 48, 24]
layers_size = [len(X_train[0])] + hidden_layers_size + [len(y_train[0])]

# Inicializo las thetas
thetas = [[] for _ in range(len(layers_size) - 1)] 
for i in range(len(thetas)):
    thetas[i] = initialize_random_weights(layers_size[i], layers_size[i + 1])
    thetas[i] = np.hstack([np.ones((len(thetas[i]), 1)), thetas[i]])

In [ ]:
lambda_ = 0.01
alpha = 0.01
iterations = 10000

print("Entrenando red...")
J_history, thetas_grad = n_converge(thetas, X_train, y_train, lambda_, alpha, iterations)
print("Red entrenada")
prediction = n_predict(thetas_grad, X_test)
print(np.mean(np.argmax(prediction, axis=1) == y_test) * 100, '%')

coefficients = []
intercepts = []

for theta in thetas_grad:
    coefficients.append(theta[:, 1:].T)
    intercepts.append(theta[:, 0]) 

## Perceptrón con sklearn

In [ ]:
from sklearn.neural_network import MLPClassifier

lambda_ = 0.01
alpha = 0.01
iterations = 10000

sklearn_model = MLPClassifier(hidden_layer_sizes=(24, 48, 24), 
                              activation='logistic', 
                              learning_rate_init=alpha, 
                              alpha=lambda_,
                              max_iter=iterations)

print("Entrenando red...")
sklearn_model.fit(X_train, y_train)
print("Red entrenada")
prediction = sklearn_model.predict(X_test)
print(np.mean(np.argmax(prediction, axis=1) == y_test) * 100, '%')

coefficients = sklearn_model.coefs_
intercepts = sklearn_model.intercepts_

thetas_grad = []

for coefficient, intercept in zip(coefficients, intercepts):
    intercept = np.array(intercept).reshape(1, -1)
    thetas_grad.append(np.vstack([intercept, coefficient]).T) 

## Parseo del modelo

In [ ]:
with open('trained_model.txt', 'w') as file:
    file.write(f"num_layers:{len(coefficients) + 1}\n")
    parameter = 0
    for coefficient, intercept in zip(coefficients, intercepts):
        file.write(f"parameter:{parameter}\n")
        file.write(f"dims:{[str(elemento) for elemento in list(coefficient.shape)]}\n")
        file.write(f"name:coefficient\n")
        file.write(f"values:[{', '.join(str(x) for x in coefficient.flatten())}]\n")
        file.write(f"parameter:{parameter}\n")
        file.write(f"dims:{[str(elemento) for elemento in list(np.array(intercept).reshape(1, -1).shape)]}\n")
        file.write(f"name:intercepts\n")
        file.write(f"values:[{', '.join(str(x) for x in np.array(intercept).flatten())}]\n")
        parameter += 1
        
print('Modelo copiado')

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Inicializar el clasificador KNN
knn = KNeighborsClassifier(n_neighbors=5, metric='manhattan')  

# Entrenar el clasificador
knn.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
prediction = knn.predict(X_test)

## Decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Crear el clasificador de árbol de decisión
tree_classifier = DecisionTreeClassifier(max_depth=10)

# Entrenar el modelo
tree_classifier.fit(X_train, y_train)

# Predecir con el conjunto de prueba
prediction = tree_classifier.predict(X_test)

## Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=500, max_depth=30, random_state=42)

print("Entrenando red...")
rf_classifier.fit(X_train, y_train)
print("Red entrenada")

prediction = rf_classifier.predict(X_test)

## Validación de los datos

In [ ]:
prediction = np.argmax(prediction, axis=1)
num_classes = 4

def confusion_matrix(y_true, y_pred, num_classes):
    matrix = [[0 for _ in range(num_classes)] for _ in range(num_classes)]

    for i in range(len(y_true)):
        true_label = y_true[i]
        pred_label = y_pred[i]
        matrix[true_label][pred_label] += 1

    return matrix

print("Confusion matrix:")
confusion_matrix = confusion_matrix(y_test, prediction, num_classes)
for row in confusion_matrix:
    print(row)
print()

def accuracy(y_true, y_pred, confusion_matrix): 
    num_classes = len(confusion_matrix)
    
    total = 0
    correct = 0
    
    for i in range(num_classes):
        for j in range(num_classes):
            total += confusion_matrix[i][j]
            if i == j:
                correct += confusion_matrix[i][j]
                
    return correct / total
                
print("Accuracy:")
print(accuracy(y_test, prediction, confusion_matrix))
print()

# ¿Tiene sentido el MSE si la salida es una variable categórica?
def mean_squared_error(y_true, y_pred):
    n = len(y_true)
    mse = sum((y_true - y_pred) ** 2) / n
    return mse

print("MSE:")
print(mean_squared_error(y_test, prediction))
print()

## Comparativa

#### KNN

Hemos elegido 5 vecinos para entrenar ya que con menos vecinos, la red tiene menos precisión, sobreajustándose peor a los valores de enternamiento y adaptándose peor a nuevos problemas; y con muchos más vecinos, la red vuelve a perder precisión ya que al estar sujeta a la "opinión" de muchos más vecinos no se entrena correctamente, incapaz de resolver los problemas. Hemos probado a utilizar diferentes funciones para realizar el cálculo de la métrica de la distancia de los puntos con los vecinos y nos hemos quedado con Manhattan porque ha sido la que mayor precisión ha dado.

#### Decision Tree

Hemos cambiado la profundidad máxima, ya que esta controla la longitud máxima del camino desde la raíz hasta las hojas. Un árbol más profundo puede aprender relaciones más complejas, pero también puede sobreajustarse fácilmente. En nuestro caso hemos visto que a partir de 10 nodos, el árbol empieza a perder precisión debido al overfitting y con menos 3 también baja, ya que el modelo no se entrena correctamente.

#### Random Forest

Hemos cambiado el número de estimadores para aumentar la cantidad de árboles en el bosque. Cuantos más árboles, más robusto suele ser el modelo. Cuantos más árboles la precisión del modelo mejora, pero también puede aumentar el tiempo de entrenamiento. También hemos cambiado la profundidad máxima de cada árbol del bosque.



Entre estos tres modelos elegiría el random forest ya que es con el que mayor precisión se consigue, por lo que me quedaría con él para entrenar la IA. Sin embargo, si buscara un modelo más rápido (y más fácil de implementar), me quedaría con el KNN, ya que no se queda por detrás en precisión y su entrenamiento es prácticamente inmediato.

## Elección de los parámetros de entrenamiento

Hemos eliminado la y, ya que el escenario es totalmente, plano, e incluso si no lo fuera, este parámetro sería también irrelevante. Preferiría guardarme la inclinación del coche respecto el eje X.

Nos hemos quedado con la x y la z ya que entrenamos el modelo para recorrer un único escenario. Si quisiéramos llevar este modelo a diferentes pistas, quitaríamos estos dos parámetros ya que serían ruido para el correcto entrenamiento del modelo.

Hemos quitado el primer rayo, ya que cuando realizamos las grabaciones, dejamos su distancia muy corta, resultando en que el rayo casi nunca cambiaba su valor respecto su máximo, significando ruido para el entrenamiento del modelo.

También hemos quitado el tiempo, ya que a la hora de poner el modelo a prueba, tomaba las curvas antes de tiempo, chocándose constantemente contra los muros. 